In [2]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization, Dropout,InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [104]:
df =pd.read_csv(r"C:\Users\syngu\OneDrive\Documents\GitHub\HandGestureWithLSTM\concated.csv")



In [105]:
labels = df["label"].unique().tolist()
for label in labels:
    df.loc[df["label"] == label, "label"] = df.loc[df["label"] == label, "label"].head(
        1170
    )

In [106]:
X = df.drop(columns=["label"])
y = df["label"]

In [ ]:
y.value_counts()

label
volume      1170
cursor      1170
ILoveYou    1170
thanks      1170
hello       1170
A           1170
B           1170
C           1170
D           1170
E           1170
F           1170
Name: count, dtype: int64

In [108]:
encoder = LabelEncoder()
window = 30
X_sequence = []
y_sequence = []
for label in df["label"].unique().tolist():
    temp_df = df.loc[df["label"]==label]
    temp_X = temp_df.drop(columns=["label"])
    for i in range(len(temp_df)-window):
        X_sequence.append(temp_X[i:i+window])
        label = temp_df['label'].iloc[i+window]
        y_sequence.append(label)


X = np.array(X_sequence)
y = np.array(y_sequence)

y = encoder.fit_transform(y)
y = to_categorical(y)

In [109]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=42)

In [110]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((11286, 30, 63), (11286, 11), (1254, 30, 63), (1254, 11))

In [111]:
model = Sequential([
    InputLayer((30,63)),
    LSTM(128),
    Dense(32,activation='relu'),
    Dense(len(df["label"].unique())-1,activation='softmax')
])

early_stopping = EarlyStopping(monitor='val_accuracy',patience=10,restore_best_weights=True)

model.compile(loss = "categorical_crossentropy", metrics =['accuracy'],optimizer = Adam(0.001))

In [112]:
model.fit(X_train,y_train,epochs=30,validation_split=0.01, batch_size=32, callbacks=[early_stopping])

Epoch 1/30
350/350 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.5998 - loss: 1.2165 - val_accuracy: 0.7611 - val_loss: 0.8982
Epoch 2/30
350/350 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9170 - loss: 0.2939 - val_accuracy: 0.9646 - val_loss: 0.1399
Epoch 3/30
350/350 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9960 - loss: 0.0271 - val_accuracy: 1.0000 - val_loss: 0.0033
Epoch 4/30
350/350 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9901 - loss: 0.0409 - val_accuracy: 1.0000 - val_loss: 0.0089
Epoch 5/30
350/350 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9816 - loss: 0.0688 - val_accuracy: 0.9912 - val_loss: 0.0190
Epoch 6/30
350/350 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9977 - loss: 0.0108 - val_accuracy: 1.0000 - val_loss: 0.0010
Epoch 7/30
350/350 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9982 - loss: 0.0068 - val_accuracy: 0.8761 - val_loss: 0.5022
Epoch 8/30
350/350 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9692 - loss: 0.1142 - val_accuracy: 

In [113]:
model.evaluate(X_test,y_test)

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.0028


[0.0029508420266211033, 1.0]

In [ ]:
model.save("test_model.keras")

: 